In [4]:
# Install required libraries
!pip install pandas openpyxl mlxtend tabulate scipy

# Step 1: Import Libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from scipy.sparse import csr_matrix
from tabulate import tabulate

# Step 2: Load Dataset
file_name = 'Online Retail.xlsx'  # Replace with your dataset file
df = pd.read_excel(file_name)

# Step 3: Data Preprocessing
df.dropna(inplace=True)
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Filter out less frequent items
item_counts = df['Description'].value_counts()
threshold = 50  # Adjust this threshold based on dataset size
frequent_items = item_counts[item_counts >= threshold].index
df = df[df['Description'].isin(frequent_items)]

print("\nFiltered Dataset (First 5 Rows):")
print(tabulate(df.head(), headers='keys', tablefmt='grid'))

# Step 4: Create Sparse Matrix
basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack(fill_value=0)
basket = basket.applymap(lambda x: 1 if x > 0 else 0)
basket_sparse = csr_matrix(basket)

print("\nBinary Transaction Matrix (First 5 Rows):")
print(tabulate(basket.iloc[:5, :5], headers='keys', tablefmt='grid'))

# Step 5: Generate Frequent Itemsets
frequent_itemsets = apriori(basket, min_support=0.02, use_colnames=True)

print("\nFrequent Itemsets (Top 5 Rows):")
print(tabulate(frequent_itemsets.head(), headers='keys', tablefmt='grid'))



/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Filtered Dataset (First 5 Rows):
+----+-------------+-------------+-------------------------------------+------------+---------------------+-------------+--------------+----------------+
|    |   InvoiceNo | StockCode   | Description                         |   Quantity | InvoiceDate         |   UnitPrice |   CustomerID | Country        |
+====+=============+=============+=====================================+============+=====================+=============+==============+================+
|  0 |      536365 | 85123A      | WHITE HANGING HEART T-LIGHT HOLDER  |          6 | 2010-12-01 08:26:00 |        2.55 |        17850 | United Kingdom |
+----+-------------+-------------+-------------------------------------+------------+---------------------+-------------+--------------+----------------+
|  1 |      536365 | 71053       | WHITE METAL LANTERN                 |          6 | 2010-12-01 08:26:00 |        3.39 |        17850 | United Kingdom |
+----+-------------+-------------+--------

<ipython-input-4-878c6e54257b>:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)



Binary Transaction Matrix (First 5 Rows):
+-------------+----------------------------------+----------------------+--------------------------------+-------------------------------+----------------------------+
|   InvoiceNo |    50'S CHRISTMAS GIFT BAG LARGE |    DOLLY GIRL BEAKER |    I LOVE LONDON MINI BACKPACK |    OVAL WALL MIRROR DIAMANTE  |    RED SPOT GIFT BAG LARGE |
+=============+==================================+======================+================================+===============================+============================+
|      536365 |                                0 |                    0 |                              0 |                             0 |                          0 |
+-------------+----------------------------------+----------------------+--------------------------------+-------------------------------+----------------------------+
|      536366 |                                0 |                    0 |                              0 |           

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(



Frequent Itemsets (Top 5 Rows):
+----+-----------+------------------------------------------------+
|    |   support | itemsets                                       |
+====+===========+================================================+
|  0 | 0.021948  | frozenset({'3 STRIPEY MICE FELTCRAFT'})        |
+----+-----------+------------------------------------------------+
|  1 | 0.0396375 | frozenset({'6 RIBBONS RUSTIC CHARM'})          |
+----+-----------+------------------------------------------------+
|  2 | 0.0254422 | frozenset({'60 CAKE CASES VINTAGE CHRISTMAS'}) |
+----+-----------+------------------------------------------------+
|  3 | 0.0358703 | frozenset({'60 TEATIME FAIRY CAKE CASES'})     |
+----+-----------+------------------------------------------------+
|  4 | 0.0273531 | frozenset({'72 SWEETHEART FAIRY CAKE CASES'})  |
+----+-----------+------------------------------------------------+


In [32]:
# Step 6: Generate Association Rules
# Ensure there are enough frequent itemsets for association rules
if not frequent_itemsets.empty:
    # Calculate the total number of itemsets to pass to association_rules
    num_itemsets = frequent_itemsets.shape[0] # Added this line
    rules = association_rules(frequent_itemsets, num_itemsets, metric="lift", min_threshold=1) # Modified this line to include num_itemsets
    print("\nAssociation Rules (Top 5 Rows):")
    print(tabulate(rules.head(), headers='keys', tablefmt='grid'))

    # Step 7: Export Results for Power BI
    rules.to_csv('association_rules.csv', index=False)
    print("\nAssociation Rules saved as 'association_rules.csv'.")

    # Step 8: Download File for Power BI
    from google.colab import files
    files.download('association_rules.csv')
else:
    print("\nNo frequent itemsets were generated with the current settings. Try reducing the min_support.")


Association Rules (Top 5 Rows):
+----+-------------------------------------------+-------------------------------------------+----------------------+----------------------+-----------+--------------+---------+--------------------+------------+--------------+-----------------+-----------+-------------+--------------+
|    | antecedents                               | consequents                               |   antecedent support |   consequent support |   support |   confidence |    lift |   representativity |   leverage |   conviction |   zhangs_metric |   jaccard |   certainty |   kulczynski |
+====+===========================================+===========================================+======================+======================+===========+==============+=========+====================+============+==============+=================+===========+=============+==============+
|  0 | frozenset({'ALARM CLOCK BAKELIKE RED '})  | frozenset({'ALARM CLOCK BAKELIKE GREEN'}) |            0.0

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>